In [1]:
using Revise

In [2]:
using SQLite
using MsgPack
using Flux
using Flux.NNlib

# Import training data from DB

In [20]:
include("gamegenerator.jl")

GameGenerator

In [21]:
db, _, _ = GameGenerator.open_db()

(SQLite.DB("games.sqlite"), SQLite.Stmt(SQLite.DB("games.sqlite"), Ptr{Void} @0x0000000005680008), SQLite.Stmt(SQLite.DB("games.sqlite"), Ptr{Void} @0x00000000062d7cb8))

In [88]:
positions = SQLite.query(db, "select positions.*, games.outcome from positions, games where positions.game_id=games.id order by random() limit 10000");

In [89]:
positions[:board_state_u] = convert(Vector{Checkers.State}, GameGenerator.unpack_state.(positions[:board_state]));
positions[:mcts_probs_u] = convert(Vector{Vector{Float32}}, MsgPack.unpack.(positions[:mcts_probs]));
positions[:mcts_moves_u] = convert(Vector{Vector{Checkers.Move}}, GameGenerator.unpack_moves.(positions[:mcts_moves]));
positions[:state_tensor] = Checkers.NN.state_to_tensor.(positions[:board_state_u]);
positions[:move_tensor] = [Checkers.NN.moves_to_tensor(p[:mcts_probs_u], p[:mcts_moves_u]) for p in DataFrames.eachrow(positions)];

# Model

## Game outcome predictor

In [90]:
?Conv

search: Conv conv conv2 Conv2D conv2d convert deconv ConjVector code_native



```
Conv(size, in=>out)
Conv(size, in=>out, relu)
```

Standard convolutional layer. `size` should be a tuple like `(2, 2)`. `in` and `out` specify the number of input and output channels respectively.

Data should be stored in WHCN order. In other words, a 100×100 RGB image would be a `100×100×3` array, and a batch of 50 would be a `100×100×3×50` array.

Takes the keyword arguments `pad` and `stride`.


In [109]:
model = Chain(
    Conv((3, 3), 8=>16, relu, pad=(1, 1)),
    Conv((3, 3), 16=>32, relu, pad=(1, 1)),
    Conv((3, 3), 32=>64, relu, pad=(1, 1)),
    Conv((3, 3), 64=>128, relu, pad=(1, 1)),
    Conv((3, 3), 128=>128, relu, pad=(1, 1)),
    Conv((3, 3), 128=>128, relu, pad=(1, 1)),
    Conv((3, 3), 128=>128, relu, pad=(1, 1)),
    Conv((3, 3), 128=>128, relu, pad=(1, 1)),
    Conv((3, 3), 128=>4, relu, pad=(1, 1)),
    x -> reshape(x, :, size(x, 4)),
    softmax
)

Chain(Conv((3, 3), 8=>16, NNlib.relu), Conv((3, 3), 16=>32, NNlib.relu), Conv((3, 3), 32=>64, NNlib.relu), Conv((3, 3), 64=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), Conv((3, 3), 128=>4, NNlib.relu), #35, NNlib.softmax)

In [110]:
loss(x, y) = Flux.crossentropy(model(x), reshape(y, :, size(y, 4)))

loss (generic function with 1 method)

In [111]:
get_batch(positions, start_idx, n) = (cat(4, positions[1:30, :state_tensor]...), cat(4, positions[1:30, :move_tensor]...))

get_batch (generic function with 1 method)

In [112]:
minibatches = [get_batch(positions, i, 1024) for i=1:1024:length(positions[1])];

In [113]:
x, y = minibatches[1];

In [114]:
loss(x, y)

4.852030290257223 (tracked)

In [118]:
opt = ADAM(params(model))

(::#71) (generic function with 1 method)

In [119]:
Flux.@epochs 50 Flux.train!(loss, minibatches, opt, cb=Flux.throttle(() -> println("Loss: ", loss(x, y)), 5))

INFO: Epoch 1


Loss: 4.505379580477796 (tracked)


INFO: Epoch 2


Loss: 3.017158944261936 (tracked)


INFO: Epoch 3


...

Loss: 2.7114281745133346 (tracked)


INFO: Epoch 49


Loss: 2.711400543161475 (tracked)


INFO: Epoch 50


Loss: 2.711409223894782 (tracked)
